In [1]:
from sagemaker.huggingface import HuggingFace
import sagemaker

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
role = sagemaker.get_execution_role()

In [3]:
role

'arn:aws:iam::730335253621:role/SageMakerLLMRole'

In [4]:
hyperparameters = {
    "model_id": "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T",
    "epochs": 2,
    "per_device_train_batch_size": 2,
    "lr": 2e-5
}

In [5]:
estimator = HuggingFace(
    entry_point="train.py",
    source_dir="./scripts",
    role=role,
    transformers_version="4.36",
    pytorch_version="2.1",
    py_version="py310",
    instance_type="ml.g5.xlarge",
    instance_count=1,
    hyperparameters=hyperparameters
)

In [6]:
estimator.fit({
    "train": "s3://llm-finetune-dataset-sunny/datasets/"
})

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-pytorch-training-2025-11-15-08-23-51-678


2025-11-15 08:23:57 Starting - Starting the training job
2025-11-15 08:23:57 Pending - Training job waiting for capacity...
2025-11-15 08:24:10 Pending - Preparing the instances for training...
2025-11-15 08:24:54 Downloading - Downloading the training image........................
2025-11-15 08:28:56 Training - Training image download completed. Training in progress...bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
/opt/conda/lib/python3.10/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/opt/conda/lib/python3.10/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms

In [7]:
estimator.model_data

's3://sagemaker-ap-south-1-730335253621/huggingface-pytorch-training-2025-11-15-08-23-51-678/output/model.tar.gz'

In [ ]:
predictor = huggingface_estimator.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.xlarge"
)

predictor.predict({"inputs": "Explain AWS S3"})
